# Estagio CD&IA - Desafio


### Importação

In [7]:
from dbfread import DBF
import numpy as np
import pandas as pd
import csv

In [8]:
# Caminho para o dbf file 
file_path = r'c:\Users\alexa\Downloads\pacigeral_jun24.dbf'

# Ler dbf file com encoding latin1
table = DBF(file_path, encoding='latin1')

In [9]:
# Arquivo novo, salvo em csv
output_csv = r'C:\Users\alexa\Downloads\pacigeral_jun24.csv'

# Abrir o arquivo CSV para escrita
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # Escrever cabeçalhos das colunas (os nomes dos campos do DBF)
    writer.writerow(table.field_names)
    
    # Escrever os registros no arquivo CSV
    for record in table:
        writer.writerow(list(record.values()))

In [10]:
# Utilizar o arquivo csv criado
csv_file_path = r'C:\Users\alexa\Downloads\pacigeral_jun24.csv'

# Carregar o arquivo CSV no Pandas
df = pd.read_csv(csv_file_path)


# Teste! 

# Exibir as primeiras linhas do DataFrame
print(df.head())

# Exibir o formato do DataFrame (número de linhas e colunas)
print(f"Formato do DataFrame: {df.shape}")

# Exibir as colunas do DataFrame
print(f"Colunas: {df.columns}")

# Exibir informações sobre os tipos de dados e valores faltantes
print(df.info())



C:\Users\alexa\AppData\Local\Temp\ipykernel_20892\572354950.py:5: DtypeWarning: Columns (37,38,39,40,77,78,79,82,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


   INSTITU                   DSCINST  ESCOLARI  IDADE  SEXO UFNASC UFRESID  \
0    22950  HOSP CLINICAS DE MARILIA         3     78     2     SP      SP   
1   612374                     ICESP         3     49     1     BA      BA   
2     9326                   UNICAMP         9     50     1     SP      SP   
3    20737              FUND PIO XII         4     33     1     SP      SP   
4     9385   SANTA CASA DE SAO PAULO         9     23     1     SP      SP   

      IBGE                CIDADE  CATEATEND  ... REC04  IBGEATEN  \
0  3534500        OSCAR BRESSANE          2  ...   NaN   3529005   
1  2930709          SIMOES FILHO          2  ...   NaN   3550308   
2  3531209   MONTE ALEGRE DO SUL          2  ...   NaN   3509502   
3  3549409  SAO JOAQUIM DA BARRA          9  ...   NaN   3505500   
4  3542602              REGISTRO          2  ...   NaN   3550308   

                     DRSINST RRASINST  CIDADEINST HABILIT  \
0           DRS 09 - Marilia  RRAS 10     MARILIA       7   


###  Seleção

#### 1. Selecionar pacientes com Topografia de pulmão  (TOPOGRUP = 34)

In [11]:
df_pulmao = df[df.TOPOGRUP.isin(['C34'])]
df_pulmao.TOPOGRUP.value_counts()
df_pulmao.shape

(56800, 105)

#### 2. Selecionar pacientes com estado de residência de São Paulo ( UFRESID = SP)

In [12]:
df_pulmao = df_pulmao[df_pulmao.UFRESID == 'SP']
df_pulmao.UFRESID.value_counts()

UFRESID
SP    52205
Name: count, dtype: int64

#### 3. Selecionar pacientes com Base do Diagnóstico com Confirmação Microscópica (BASEDIAG = 3)

In [13]:
df_pulmao = df_pulmao[df_pulmao.BASEDIAG == 3]
df_pulmao.BASEDIAG.value_counts()

BASEDIAG
3    50583
Name: count, dtype: int64

#### 4. Retirar categorias 0, X e Y da coluna ECGRUP

In [14]:
df_pulmao = df_pulmao[~df_pulmao.ECGRUP.isin(['0', 'X', 'Y'])]
df_pulmao.ECGRUP.value_counts()

ECGRUP
IV     25611
III    11972
I       4864
II      2892
Name: count, dtype: int64

#### 5. Retirar pacientes que fizeram Hormonioterapia e TMO - TMO == 1 HORMONIO = 1

In [15]:
df_pulmao = df_pulmao[~((df_pulmao['HORMONIO'] == 1) | (df_pulmao['TMO'] == 1))]
print(df_pulmao.shape)


(45128, 105)


#### 6. Selecionar pacientes com Ano de Diagnóstico até 2019 (ANODIAG ¡= 2019)

In [16]:
df_pulmao = df_pulmao[~(df_pulmao.ANODIAG > 2019)]
df_pulmao.ANODIAG.value_counts().sort_index()

ANODIAG
2000    1382
2001    1367
2002    1474
2003    1571
2004    1693
2005    1811
2006    1633
2007    1569
2008    1725
2009    1963
2010    1950
2011    2073
2012    2098
2013    2306
2014    2412
2015    2346
2016    2439
2017    2410
2018    2261
2019    2199
Name: count, dtype: int64

#### 7. Retirar pacientes com IDADE menor do que 20 anos

In [17]:
df_pulmao = df_pulmao[df_pulmao.IDADE >= 20]
df_pulmao.IDADE.value_counts()
df_pulmao.shape

(38670, 105)

### Ajustes

#### 8. Calcular a diferen¸ca em dias entre Diagnóstico e Consulta, Tratamento e Diagnóstico,
Tratamento e Consulta, a partir dessas 3 datas (DTCONSULT, DTDIAG e DTTRAT).
Após o cálculo, codificar as colunas da seguinte forma:

• CONSDIAG – 0 = até 30 dias; 1 = entre 31 e 60 dias; 2 = mais de 61 dias;
1

• DIAGTRAT – 0 = até 60 dias; 1 = entre 61 e 90 dias; 2 = mais de 91 dias; 3 = Não tratou (datas de tratamento vazias);

• TRATCONS – 0 = at´e 60 dias; 1 = entre 61 e 90 dias; 2 = mais de 91 dias; 3 = Não tratou (datas de tratamento vazias).

In [18]:
list_datas = ['DTCONSULT', 'DTDIAG', 'DTTRAT']

for col_data in list_datas:
    df_pulmao[col_data] = pd.to_datetime(df_pulmao[col_data])

df_pulmao[list_datas].dtypes

DTCONSULT    datetime64[ns]
DTDIAG       datetime64[ns]
DTTRAT       datetime64[ns]
dtype: object

In [19]:
df_pulmao['CONSDIAG'] = (df_pulmao.DTDIAG - df_pulmao.DTCONSULT).dt.days
df_pulmao['DIAGTRAT'] = (df_pulmao.DTTRAT - df_pulmao.DTDIAG).dt.days
df_pulmao['TRATCONS'] = (df_pulmao.DTTRAT - df_pulmao.DTCONSULT).dt.days

In [20]:
# testar quando tem Nan, para poder conferir se os valores de 3 estao corretos 
df_pulmao[['CONSDIAG','DIAGTRAT', 'TRATCONS']].isna().sum()

CONSDIAG       0
DIAGTRAT    6390
TRATCONS    6390
dtype: int64

In [21]:
df_pulmao['CONSDIAG_CAT'] = [0 if consdiag <= 30 else 1 if consdiag <= 60 else 2 for consdiag in df_pulmao.CONSDIAG]

df_pulmao['DIAGTRAT_CAT'] = [0 if diagtrat <= 60 else 1 if diagtrat <= 90 else 2 if diagtrat >= 91 else 3 for diagtrat in df_pulmao.DIAGTRAT]

df_pulmao['TRATCONS_CAT'] = [0 if tratcons <= 60 else 1 if tratcons <=90 else 2 if tratcons >= 91 else 3 for tratcons in df_pulmao.TRATCONS]

print(df_pulmao['CONSDIAG_CAT'].value_counts())
print(df_pulmao['DIAGTRAT_CAT'].value_counts())
print(df_pulmao['TRATCONS_CAT'].value_counts())


CONSDIAG_CAT
0    32637
1     3413
2     2620
Name: count, dtype: int64
DIAGTRAT_CAT
0    23552
3     6390
2     4718
1     4010
Name: count, dtype: int64
TRATCONS_CAT
0    22568
3     6390
2     5582
1     4130
Name: count, dtype: int64


#### 9. Extrair somente o número das colunas DRS e DRSINST ( não achei DRSINSTITU nos dados)

In [22]:
print(df_pulmao.DRS.unique())
print(df_pulmao.DRSINST.unique())

['DRS 10  Piracicaba' 'DRS 03  Araraquara' 'DRS 01  Grande Sao Paulo'
 'DRS 07  Campinas' 'DRS 13  Ribeirao Preto' 'DRS 17  Taubate'
 'DRS 15  Sao Jose do Rio Preto' 'DRS 16  Sorocaba' 'DRS 09  Marilia'
 'DRS 04  Baixada Santista' 'DRS 06  Bauru' 'DRS 02  Aracatuba'
 'DRS 11  Presidente Prudente' 'DRS 08  Franca' 'DRS 05  Barretos'
 'DRS 14  Sao Joao da Boa Vista' 'DRS 12  Registro']
['DRS 10 - Piracicaba' 'DRS 03 - Araraquara' 'DRS 01 - Grande Sao Paulo'
 'DRS 07 - Campinas' 'DRS 13 - Ribeirao Preto' 'DRS 17 - Taubate'
 'DRS 15 - Sao Jose do Rio Preto' 'DRS 16 - Sorocaba' 'DRS 09 - Marilia'
 'DRS 04 - Baixada Santista' 'DRS 06 - Bauru' 'DRS 02 - Aracatuba'
 'DRS 11 - Presidente Prudente' 'DRS 05 - Barretos' 'DRS 08 - Franca'
 'DRS 12 - Registro' 'DRS 14 - Sao Joao da Boa Vista']


In [23]:
drs_expand = df_pulmao.DRS.str.split(' ', expand=True)
df_pulmao['nDRS'] = drs_expand[1].astype(int)
df_pulmao['nDRS'].value_counts()

nDRS
1     18158
15     2715
7      2684
6      2563
13     1754
10     1473
17     1294
9      1212
3      1167
16     1045
5      1019
2       850
8       718
14      711
4       700
11      532
12       75
Name: count, dtype: int64

In [24]:
drs_expand = df_pulmao.DRS.str.split(' ', expand=True)
df_pulmao['nDRSINST'] = drs_expand[1].astype(int)
df_pulmao['nDRSINST'].value_counts()

nDRSINST
1     18158
15     2715
7      2684
6      2563
13     1754
10     1473
17     1294
9      1212
3      1167
16     1045
5      1019
2       850
8       718
14      711
4       700
11      532
12       75
Name: count, dtype: int64

### Criação

#### 10. Criar a coluna binária de óbito, a partir da coluna ULTINFO, onde as categorias 1 e 2 representam que o paciente está vivo e as 3 e 4 representam o óbito por qualquer motivo

In [25]:
# para conferir os valores de cada informacao com o obtido a seguir, classificando com 0 e 1
print((df_pulmao['ULTINFO']).value_counts())

ULTINFO
3    31160
4     3760
2     2285
1     1465
Name: count, dtype: int64


In [26]:
df_pulmao['obito'] = [0 if ultinfo in [1, 2] else 1 for ultinfo in df_pulmao['ULTINFO']]

print(df_pulmao['obito'].value_counts())

obito
1    34920
0     3750
Name: count, dtype: int64


### Retirada de Colunas

##### 11. Retirar as colunas:
UFNASC, UFRESID, CIDADE, DTCONSULT, CLINICA , DTDIAG,
BASEDIAG, TOPOGRUP, DESCTOPO, DESCMORFO, T, N, M, PT, PN,
PM, S , G, LOCALTNM, IDMITOTIC, PSA, GLEASON, OUTRACLA,
META01, META02, META03, META04, DTTRAT, NAOTRAT,
TRATAMENTO, TRATHOSP, TRATFANTES, TRATFAPOS, HORMONIO,
TMO, NENHUMANT, CIRURANT, RADIOANT, QUIMIOANT, HORMOANT,
TMOANT, IMUNOANT, OUTROANT, HORMOAPOS, TMOAPOS, DTULTINFO,
CICI , CICIGRUP, CICISUBGRU, FAIXAETAR, LATERALI, INSTORIG,
RRAS, ERRO, DTRECIDIVA, RECNENHUM, RECLOCAL, RECREGIO,
RECDIST, REC01, REC02, REC03, REC04, CIDO, DSCCIDO,
HABILIT , HABIT11 , HABILIT1 , CIDADEH, PERDASEG


In [27]:
list_drop = [ 'UFNASC', 'UFRESID', 'CIDADE', 'DTCONSULT', 'CLINICA', 'DTDIAG', 'BASEDIAG', 'TOPOGRUP', 'DESCTOPO', 'DESCMORFO', 'T', 'N', 'M', 'PT', 'PN', 'PM', 'S', 'G', 'LOCALTNM', 'IDMITOTIC', 'PSA',  'GLEASON', 'OUTRACLA', 'META01', 'META02', 'META03', 'META04', 'DTTRAT', 'NAOTRAT', 'TRATAMENTO', 'TRATHOSP', 'TRATFANTES', 'TRATFAPOS', 'HORMONIO', 'TMO', 'NENHUMANT', 'CIRURANT', 'RADIOANT', 'QUIMIOANT', 'HORMOANT', 'TMOANT', 'IMUNOANT', 'OUTROANT', 'HORMOAPOS', 'TMOAPOS', 'DTULTINFO', 'CICI', 'CICIGRUP', 'CICISUBGRU', 'FAIXAETAR', 'LATERALI', 'INSTORIG', 'RRAS', 'ERRO', 'DTRECIDIVA', 'RECNENHUM', 'RECLOCAL', 'RECREGIO', 'RECDIST', 'REC01', 'REC02', 'REC03', 'REC04', 'CIDO', 'DESCIDO', 'HABILIT', 'HABIT11', 'HABILIT1', 'CIDADEH', 'PERDASEG']

df_pulmao = df_pulmao.drop(columns=list_drop)

print(df_pulmao.shape)
df_pulmao.head(3)

(38670, 41)


,INSTITU,DSCINST,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,MORFO,...,DRSINST,RRASINST,CIDADEINST,HABILIT2,CONSDIAG_CAT,DIAGTRAT_CAT,TRATCONS_CAT,nDRS,nDRSINST,obito
169,20230,SANTA CASA DE ARARAS,9,73,1,3503307,9,2,C340,82513,...,DRS 10 - Piracicaba,RRAS 14,ARARAS,1,0,0,0,10,10,1
171,20621,SANTA CASA DE ARARAQUARA,9,67,2,3529302,9,1,C340,82513,...,DRS 03 - Araraquara,RRAS 18,ARARAQUARA,1,0,0,0,3,3,0
172,18058,HOSP ANCHIETA FUNDACAO ABC,2,72,2,3548708,2,2,C340,82513,...,DRS 01 - Grande Sao Paulo,RRAS 01,SAO BERNARDO DO CAMPO,1,0,1,0,1,1,1


In [28]:
df_pulmao.shape

(38670, 41)

### Pré Processamento 

In [ ]:
df_pulmao.describe().T

In [30]:
df_pulmao.columns

Index(['INSTITU', 'DSCINST', 'ESCOLARI', 'IDADE', 'SEXO', 'IBGE', 'CATEATEND',
       'DIAGPREV', 'TOPO', 'MORFO', 'EC', 'ECGRUP', 'NENHUM', 'CIRURGIA',
       'RADIO', 'QUIMIO', 'IMUNO', 'OUTROS', 'NENHUMAPOS', 'CIRURAPOS',
       'RADIOAPOS', 'QUIMIOAPOS', 'IMUNOAPOS', 'OUTROAPOS', 'ULTINFO',
       'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG', 'DRS', 'IBGEATEN',
       'DRSINST', 'RRASINST', 'CIDADEINST', 'HABILIT2', 'CONSDIAG_CAT',
       'DIAGTRAT_CAT', 'TRATCONS_CAT', 'nDRS', 'nDRSINST', 'obito'],
      dtype='object')

#### Divisão Treino e Teste

In [91]:
x = df_pulmao.drop(columns=['DSCINST', 'DRSINST', 'MORFO', 'ULTINFO', 'CIDADEINST','DRSINST', 'ECGRUP', 'TRATCONS', 'DIAGTRAT', 'CONSDIAG', 'NENHUMAPOS', 'CIRURAPOS', 'RADIOAPOS', 'QUIMIOAPOS', 'IMUNOAPOS', 'OUTROAPOS', 'DRS', 'RRASINST' ])

y = df_pulmao['obito']

In [92]:
from sklearn.model_selection import train_test_split

# 25% dos dados reservados para o conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state=5)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((29002, 24), (9668, 24), (29002,), (9668,))

In [93]:
# Teste para verificar se a divisao foi feita corretamente
print(f"Média idade {df_pulmao['IDADE'].mean()}")
print(f"Treino média idade {x_train['IDADE'].mean()}")
print(f"Teste média idade {x_test['IDADE'].mean()}")

Média idade 63.70170674941815
Treino média idade 63.68126336114751
Teste média idade 63.763032685146875


##### Codificação Ordinal

In [94]:
# Identificação de colunas categóricas
x_train.select_dtypes(include='object').columns

Index(['TOPO', 'EC'], dtype='object')

In [95]:
# Informações contidas na coluna
np.sort(x_train.TOPO.unique())

array(['C340', 'C341', 'C342', 'C343', 'C348', 'C349'], dtype=object)

In [96]:
np.sort(x_train.EC.unique())

array(['IA', 'IB', 'IIA', 'IIB', 'IIIA', 'IIIB', 'IV'], dtype=object)

In [97]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=-1)
oe.fit(x_train[['EC']])

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [98]:
# Aplica codificação ordinal na coluna EC
x_train[['EC']] = oe.transform(x_train[['EC']])
x_train[['EC']]

,EC
888679,6.0
885634,6.0
581799,6.0
177162,6.0
929761,5.0
...,...
181576,5.0
1010176,5.0
576777,6.0
571356,6.0


In [99]:
# Categorias da coluna que foi "modificada"
oe.categories_

[array(['IA', 'IB', 'IIA', 'IIB', 'IIIA', 'IIIB', 'IV'], dtype=object)]

In [100]:
# Identificação de colunas categóricas
x_train.select_dtypes(include='object').columns

Index(['TOPO'], dtype='object')

#####  One Hot Encoder

In [102]:
from sklearn.preprocessing import OneHotEncoder

# Garantir que o original de treino não seja modificado
x_train_ohe = x_train.copy()

# Pode eliminar a primeira coluna, pois ela é redundante
ohe = OneHotEncoder(handle_unknown='ignore', drop='first')

# Prepara categorias da coluna TOPO para codificação one-hot
ohe.fit(x_train_ohe[['TOPO']])

OneHotEncoder(drop='first', handle_unknown='ignore')

In [103]:
# Aplica one-hot na coluna TOPO, transformando os valores categoricos em binários
ohe_results = ohe.transform(x_train_ohe[['TOPO']]).toarray()
ohe_results[:4]

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

In [104]:
# Cria um DataFrame com os resultados da codificação one-hot com mesmos indices para "juntar"
# Nome das colunas: nome original ['TOPO'] e nome da categoria
df_aux = pd.DataFrame(ohe_results, index=x_train_ohe.index,
                      columns=ohe.get_feature_names_out())
df_aux.head(3)

,TOPO_C341,TOPO_C342,TOPO_C343,TOPO_C348,TOPO_C349
888679,0.0,0.0,0.0,0.0,1.0
885634,1.0,0.0,0.0,0.0,0.0
581799,0.0,0.0,0.0,0.0,1.0


In [105]:
# Associar original com o Dataframe que possui os dados após one-hot 
# Left: mantém a original fixa na esquerda e associa a auxiliar ao lado 
x_train_ohe = x_train_ohe.merge(df_aux, how='left', left_index=True,
                                right_index=True)

# Eliminar a coluna 'TOPO' pois já foi codificada
x_train_ohe.drop(columns=['TOPO'], inplace=True)
x_train_ohe.head(3)

,INSTITU,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,EC,NENHUM,CIRURGIA,...,DIAGTRAT_CAT,TRATCONS_CAT,nDRS,nDRSINST,obito,TOPO_C341,TOPO_C342,TOPO_C343,TOPO_C348,TOPO_C349
888679,612374,2,60,1,3546801,9,1,6.0,0,0,...,0,1,1,1,1,0.0,0.0,0.0,0.0,1.0
885634,16675,5,62,1,3550308,1,2,6.0,0,0,...,2,0,1,1,1,1.0,0.0,0.0,0.0,0.0
581799,9385,9,67,1,3550308,2,1,6.0,0,1,...,0,0,1,1,1,0.0,0.0,0.0,0.0,1.0


In [106]:
# Não existem mais colunas categóricas
x_train_ohe.select_dtypes(include='object').columns

Index([], dtype='object')

#####  Normalização

In [107]:
from sklearn.preprocessing import MinMaxScaler

# Dados de treino para normalização, manter entre 0 e 1
mms = MinMaxScaler()
mms.fit(x_train_ohe)

MinMaxScaler()

In [108]:
x_train_norm = mms.transform(x_train_ohe)
x_train_norm

array([[0.82742863, 0.125     , 0.51282051, ..., 0.        , 0.        ,
        1.        ],
       [0.0225257 , 0.5       , 0.53846154, ..., 0.        , 0.        ,
        0.        ],
       [0.01267552, 1.        , 0.6025641 , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.0125958 , 0.125     , 0.62820513, ..., 0.        , 0.        ,
        0.        ],
       [0.02588341, 0.375     , 0.52564103, ..., 0.        , 0.        ,
        1.        ],
       [0.02296484, 0.375     , 0.07692308, ..., 1.        , 0.        ,
        0.        ]])

In [109]:
x_train_norm.min()

0.0

In [110]:
x_train_norm.max()

1.0

#####  Transformar dados de Teste para ficarem padronizados com o de treino

###### Comandos que foram executados na parte de treino 

In [111]:
# Verificar colunas categóricas
x_test.select_dtypes(include='object').columns

Index(['TOPO', 'EC'], dtype='object')

In [112]:
# Garantir que não irá alterar dados originais
x_test_enc = x_test.copy()

# Ordinal Encoder
x_test_enc[['EC']] = oe.transform(x_test_enc[['EC']])
x_test_enc[['EC']]

,EC
578409,6.0
930398,4.0
962114,6.0
959444,6.0
183036,1.0
...,...
187502,1.0
33822,6.0
882037,6.0
33935,6.0


In [113]:
# Verificar colunas categóricas
x_test_enc.select_dtypes(include='object').columns

Index(['TOPO'], dtype='object')

In [114]:
# One Hot Encoder
ohe_results = ohe.transform(x_test_enc[['TOPO']]).toarray()
df_aux = pd.DataFrame(ohe_results, index=x_test_enc.index,
                      columns=ohe.get_feature_names_out())
x_test_enc = x_test_enc.merge(df_aux, how='left', left_index=True,
                              right_index=True)

x_test_enc.drop(columns=['TOPO'], inplace=True)
x_test_enc.head(3)


,INSTITU,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,EC,NENHUM,CIRURGIA,...,DIAGTRAT_CAT,TRATCONS_CAT,nDRS,nDRSINST,obito,TOPO_C341,TOPO_C342,TOPO_C343,TOPO_C348,TOPO_C349
578409,38628,2,78,1,3512506,9,2,6.0,0,0,...,0,0,2,2,1,0.0,0.0,0.0,0.0,1.0
930398,16624,4,62,1,3550308,9,2,4.0,0,0,...,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0
962114,21423,9,48,1,3506003,9,1,6.0,0,1,...,0,0,6,6,1,0.0,0.0,0.0,0.0,1.0


In [115]:
# Verificar colunas categóricas
x_test_enc.select_dtypes(include='object').columns

Index([], dtype='object')

In [117]:
# Normalização
x_test_norm = mms.transform(x_test_enc)

In [118]:
# Conferir treino e teste, devem ter mesma quantidade de colunas e linhas conforme a divisão anterior
x_test_norm.shape, x_train_norm.shape

((9668, 28), (29002, 28))